# 데이터 다루기

In [1]:
import numpy as np
import pandas as pd

### 1. CCTV 데이터

In [2]:
# 1) 데이터 파일 읽기 
filename = '서울시CCTV설치운영현황(자치구)_년도별_211231기준.csv'
cctv = pd.read_csv(filename, skiprows=1, encoding='euc-kr')     # thousands=','
cctv.head(3)

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
0,계,"83,557","4,812","1,851","3,434","4,295","6,840","8,708","11,572","10,627","12,267","11,247","7,904"
1,종로구,"1,715",815,NaN,NaN,195,150,0,261,85,9,200,0
2,중 구,"2,447",16,114,87,77,236,240,372,386,155,361,403


In [3]:
# 2) Null data 있는지 확인하고 0으로 대체하기 
cctv.isna().sum()

구분          0
총계          0
2012년 이전    0
2012년       1
2013년       1
2014년       0
2015년       0
2016년       0
2017년       0
2018년       0
2019년       0
2020년       0
2021년       0
dtype: int64

In [4]:
cctv['2012년'].fillna('0', inplace=True)
cctv['2013년'].fillna('0', inplace=True)

In [5]:
# 3) 1000 단위 구분기호 없애고 정수로 변환하기 
cctv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   구분        26 non-null     object
 1   총계        26 non-null     object
 2   2012년 이전  26 non-null     object
 3   2012년     26 non-null     object
 4   2013년     26 non-null     object
 5   2014년     26 non-null     object
 6   2015년     26 non-null     object
 7   2016년     26 non-null     object
 8   2017년     26 non-null     object
 9   2018년     26 non-null     object
 10  2019년     26 non-null     object
 11  2020년     26 non-null     object
 12  2021년     26 non-null     object
dtypes: object(13)
memory usage: 2.8+ KB


In [6]:
for column in cctv.columns[1:]:
    cctv[column] = cctv[column].str.replace(',','').astype(int)
    #cctv[column] = cctv[column].apply(lambda x: int(x.replace(',','')))
cctv.head()

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
0,계,83557,4812,1851,3434,4295,6840,8708,11572,10627,12267,11247,7904
1,종로구,1715,815,0,0,195,150,0,261,85,9,200,0
2,중 구,2447,16,114,87,77,236,240,372,386,155,361,403
3,용산구,2611,34,71,234,125,221,298,351,125,307,617,228
4,성동구,3829,163,144,208,107,325,255,967,415,490,472,283


In [7]:
# 4) 최근 3개년간의 CCTV 증가율 구하기 
cctv['2017년_이전'] = cctv.총계 - cctv['2018년'] - cctv['2019년'] - cctv['2020년'] - cctv['2021년']
cctv['최근증가율'] = ((cctv['2018년']+cctv['2019년']+cctv['2020년']) / cctv['2017년_이전'] * 100).round(2)
cctv.head(3)

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2017년_이전,최근증가율
0,계,83557,4812,1851,3434,4295,6840,8708,11572,10627,12267,11247,7904,41512,82.24
1,종로구,1715,815,0,0,195,150,0,261,85,9,200,0,1421,20.69
2,중 구,2447,16,114,87,77,236,240,372,386,155,361,403,1142,78.98


In [8]:
cctv['2017년이전2'] = cctv.iloc[:, 2:9].sum(axis=1)
cctv.head(3)

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2017년_이전,최근증가율,2017년이전2
0,계,83557,4812,1851,3434,4295,6840,8708,11572,10627,12267,11247,7904,41512,82.24,41512
1,종로구,1715,815,0,0,195,150,0,261,85,9,200,0,1421,20.69,1421
2,중 구,2447,16,114,87,77,236,240,372,386,155,361,403,1142,78.98,1142


In [9]:
# 5) 구분, 총계, 최근증가율 컬럼만 추출
cctv = cctv[['구분', '총계','최근증가율']]

In [10]:
# 6) 컬럼명을 구별, CCTV댓수로 변경하기 
cctv.rename(columns={'구분':'구별', '총계':'CCTV댓수'}, inplace=True)
cctv.head(3)

,구별,CCTV댓수,최근증가율
0,계,83557,82.24
1,종로구,1715,20.69
2,중 구,2447,78.98


In [11]:
# 7) 구 이름에서 공백지우기 
cctv.구별 = cctv.구별.str.replace(' ','')
cctv.head(3)

,구별,CCTV댓수,최근증가율
0,계,83557,82.24
1,종로구,1715,20.69
2,중구,2447,78.98


In [12]:
# 8) 첫번째 행 지우기
cctv.drop([0], inplace=True)
cctv.head(3)

,구별,CCTV댓수,최근증가율
1,종로구,1715,20.69
2,중구,2447,78.98
3,용산구,2611,78.64


### 2. 서울시 인구 데이터

In [13]:
# 1) 데이터 파일 읽기
filename = '서울시인구.txt'
pop = pd.read_csv(filename, sep='\t', skiprows=2)
pop.head(3)

,기간,자치구,세대,계,남자,여자,계.1,남자.1,여자.1,계.2,남자.2,여자.2,세대당인구,65세이상고령자
0,2021,합계,"4,426,007","9,736,027","4,721,977","5,014,050","9,509,458","4,618,040","4,891,418","226,569","103,937","122,632",2.15,"1,605,416"
1,2021,종로구,"73,494","153,789","74,186","79,603","144,683","70,183","74,500","9,106","4,003","5,103",1.97,"27,818"
2,2021,중구,"63,519","131,787","64,083","67,704","122,499","59,630","62,869","9,288","4,453","4,835",1.93,"24,392"


In [14]:
# 2) Null data 있는지 확인하기 
pop.isna().sum()

기간          0
자치구         0
세대          0
계           0
남자          0
여자          0
계.1         0
남자.1        0
여자.1        0
계.2         0
남자.2        0
여자.2        0
세대당인구       0
65세이상고령자    0
dtype: int64

In [15]:
# 3) '자치구', '계', '계.1', '계.2', '65세이상고령자‘ 컬럼만 추출하기
pop = pop[['자치구','계','계.1','계.2','65세이상고령자']]
pop.head(3)

,자치구,계,계.1,계.2,65세이상고령자
0,합계,"9,736,027","9,509,458","226,569","1,605,416"
1,종로구,"153,789","144,683","9,106","27,818"
2,중구,"131,787","122,499","9,288","24,392"


In [16]:
# 4) 컬럼명을 '구별', '인구수', '내국인', '외국인', '고령자'로 변경하기
pop.columns = ['구별','인구수','내국인','외국인','고령자']
pop.head(3)

,구별,인구수,내국인,외국인,고령자
0,합계,"9,736,027","9,509,458","226,569","1,605,416"
1,종로구,"153,789","144,683","9,106","27,818"
2,중구,"131,787","122,499","9,288","24,392"


In [17]:
# 5) 1000 단위 구분기호 없애고 정수로 변환하기 
for column in pop.columns[1:]:
    pop[column] = pop[column].str.replace(',','').astype(int)
    # pop[column] = pop[column].apply(lambda x: int(x.replace(',','')))
pop.head(3)

,구별,인구수,내국인,외국인,고령자
0,합계,9736027,9509458,226569,1605416
1,종로구,153789,144683,9106,27818
2,중구,131787,122499,9288,24392


In [18]:
# 6) 첫번째 행(합계행) 지우기 
pop.drop(0, inplace=True)
pop.head(3)

,구별,인구수,내국인,외국인,고령자
1,종로구,153789,144683,9106,27818
2,중구,131787,122499,9288,24392
3,용산구,237285,222953,14332,39070


In [19]:
# 7) '외국인비율'과 '고령자비율' 컬럼 만들기
pop['외국인비율'] = (pop.외국인/pop.인구수*100).round(2)
pop['고령자비율'] = np.round(pop.고령자/pop.인구수*100, 2)
pop.head(3)

,구별,인구수,내국인,외국인,고령자,외국인비율,고령자비율
1,종로구,153789,144683,9106,27818,5.92,18.09
2,중구,131787,122499,9288,24392,7.05,18.51
3,용산구,237285,222953,14332,39070,6.04,16.47


In [20]:
# 8) 구 이름에 공백이 있으면 공백지우기
pop.구별 = pop.구별.str.replace(' ','')
pop.head(3)

,구별,인구수,내국인,외국인,고령자,외국인비율,고령자비율
1,종로구,153789,144683,9106,27818,5.92,18.09
2,중구,131787,122499,9288,24392,7.05,18.51
3,용산구,237285,222953,14332,39070,6.04,16.47


### 3. CCTV 데이터와 인구 데이터 합치기

In [21]:
# 1) 문항 1, 2에서 만든 데이터 합치기
df = pd.merge(cctv, pop)
df.head(3)

,구별,CCTV댓수,최근증가율,인구수,내국인,외국인,고령자,외국인비율,고령자비율
0,종로구,1715,20.69,153789,144683,9106,27818,5.92,18.09
1,중구,2447,78.98,131787,122499,9288,24392,7.05,18.51
2,용산구,2611,78.64,237285,222953,14332,39070,6.04,16.47


In [22]:
# 2) '구별' 컬럼을 인덱스로 만들기
df.set_index('구별', inplace=True)
df.head(3)

,CCTV댓수,최근증가율,인구수,내국인,외국인,고령자,외국인비율,고령자비율
구별,,,,,,,,
종로구,1715,20.69,153789,144683,9106,27818,5.92,18.09
중구,2447,78.98,131787,122499,9288,24392,7.05,18.51
용산구,2611,78.64,237285,222953,14332,39070,6.04,16.47


In [23]:
# 3) 합친 데이터를 cctv_pop.csv로 저장하기
df.to_csv('cctv_pop.csv')

In [25]:
# 4) 검증 차원에서 cctv_pop.csv에서 데이터 읽어서 확인하기
vf = pd.read_csv('cctv_pop.csv', index_col='구별')
vf.head(3)

,CCTV댓수,최근증가율,인구수,내국인,외국인,고령자,외국인비율,고령자비율
구별,,,,,,,,
종로구,1715,20.69,153789,144683,9106,27818,5.92,18.09
중구,2447,78.98,131787,122499,9288,24392,7.05,18.51
용산구,2611,78.64,237285,222953,14332,39070,6.04,16.47
